In [1]:
! nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3060 (UUID: GPU-9ef79948-0acd-05f8-e2b4-18b37aa1f16a)


# Installs

In [ ]:
%%time

from IPython.display import clear_output

! pip install sentence_transformers==2.2.2

! pip install -qq -U langchain
! pip install -qq -U tiktoken
! pip install -qq -U pypdf
! pip install -qq -U faiss-gpu
! pip install -qq -U InstructorEmbedding 

! pip install -qq -U transformers 
! pip install -qq -U accelerate
! pip install -qq -U bitsandbytes

clear_output()

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# Imports

In [2]:
%%time

import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

### loaders
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

### splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

### prompts
from langchain import PromptTemplate, LLMChain

### vector stores
from langchain.vectorstores import FAISS

### models
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings

### retrievers
from langchain.chains import RetrievalQA

import torch
import transformers
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

CPU times: total: 3.86 s
Wall time: 9.01 s


In [3]:
print('langchain:', langchain.__version__)
print('torch:', torch.__version__)
print('transformers:', transformers.__version__)

langchain: 0.2.15
torch: 2.2.2
transformers: 4.44.2


In [4]:
sorted(glob.glob('Dataset2/*'))

['Dataset2\\myQA.docx', 'Dataset2\\myQA.pdf']

# CFG

- CFG class enables easy and organized experimentation 

In [5]:
class CFG:
    # LLMs
    model_name = 'Persian' # wizardlm, llama2-7b-chat, llama2-13b-chat, mistral-7B
    temperature = 0
    top_p = 0.95
    repetition_penalty = 1.15    

    # splitting
    split_chunk_size = 800
    split_overlap = 0
    
    # embeddings
    embeddings_model_repo = 'sentence-transformers/all-MiniLM-L6-v2'    

    # similar passages
    k = 6
    
    # paths
    PDFs_path = './Dataset2'
    Embeddings_path =  './faiss-hp-sentence-transformers'
    Output_folder = './qa-vectordb'

# Define model

In [6]:
def get_model(model = CFG.model_name):

    print('\nDownloading model: ', model, '\n\n')

    if model == 'wizardlm':
        model_repo = 'TheBloke/wizardLM-7B-HF'
        
        tokenizer = AutoTokenizer.from_pretrained(model_repo)
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type = "nf4",
            bnb_4bit_compute_dtype = torch.float16,
            bnb_4bit_use_double_quant = True,
        )        

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            quantization_config = bnb_config,
            device_map = 'auto',
            low_cpu_mem_usage = True
        )
        
        max_len = 1024

    elif model == 'llama2-7b-chat':
        model_repo = 'daryl149/llama-2-7b-chat-hf'
        
        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type = "nf4",
            bnb_4bit_compute_dtype = torch.float16,
            bnb_4bit_use_double_quant = True,
        )
        
        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            quantization_config = bnb_config,
            device_map = 'auto',
            low_cpu_mem_usage = True,
            trust_remote_code = True
        )
        
        max_len = 2048

    elif model == 'llama2-13b-chat':
        model_repo = 'daryl149/llama-2-13b-chat-hf'
        
        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type = "nf4",
            bnb_4bit_compute_dtype = torch.float16,
            bnb_4bit_use_double_quant = True,
        )
                
        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            quantization_config = bnb_config,       
            device_map = 'auto',
            low_cpu_mem_usage = True,
            trust_remote_code = True
        )
        
        max_len = 2048 # 8192

    elif model == 'mistral-7B':
        model_repo = 'mistralai/Mistral-7B-v0.1'
        
        tokenizer = AutoTokenizer.from_pretrained(model_repo)
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type = "nf4",
            bnb_4bit_compute_dtype = torch.float16,
            bnb_4bit_use_double_quant = True,
        )        

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            quantization_config = bnb_config,
            device_map = 'auto',
            low_cpu_mem_usage = True,
        )
        
        max_len = 1024
        
    elif model == 'Persian':
        model_repo = "MehdiHosseiniMoghadam/AVA-Llama-3-V2"
        
        # model = AutoModelForCausalLM.from_pretrained(model_repo, torch_dtype=torch.float16, low_cpu_mem_usage=True, load_in_8bit=True)
        tokenizer = AutoTokenizer.from_pretrained(model_repo)
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type = "nf4",
            bnb_4bit_compute_dtype = torch.float16,
            bnb_4bit_use_double_quant = True,
        )        

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            quantization_config = bnb_config,
            device_map = 'auto',
            low_cpu_mem_usage = True,
        )
        
        max_len = 2048
    else:
        print("Not implemented model (tokenizer and backbone)")

    return tokenizer, model, max_len

In [7]:
%%time

tokenizer, model, max_len = get_model(model = CFG.model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 9.52 s
Wall time: 16.8 s


In [8]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128258, 4096, padding_idx=128257)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Llama

In [9]:
### check how Accelerate split the model across the available devices (GPUs)
model.hf_device_map

{'': 0}

# pipeline

- Hugging Face pipeline

In [10]:
### hugging face pipeline
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    pad_token_id = tokenizer.eos_token_id,
#     do_sample = True,
    max_length = max_len,
    temperature = CFG.temperature,
    top_p = CFG.top_p,
    repetition_penalty = CFG.repetition_penalty
)

### langchain pipeline
llm = HuggingFacePipeline(pipeline = pipe)

C:\Users\a.nasimi\Anaconda3\envs\LLM_Env\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [11]:
%%time
### testing model, not using the harry potter books yet
### answer is not necessarily related to harry potter
query = "سلام خوبی"
llm.invoke(query)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


CPU times: total: 2.45 s
Wall time: 2.89 s


'سلام خوبی است. در اینجا یک پاسخ به سؤال شما ارائه می شود:\n\nپاسخ: بله، استفاده از کرم صورت برای پوست خشک و حساس مفید است.'

# Langchain

- Multiple document retriever with LangChain

In [12]:
CFG.model_name

'Persian'

# Loader

In [13]:
%%time

loader = DirectoryLoader(
    CFG.PDFs_path,
    glob="./*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True,
    use_multithreading=True
)

documents = loader.load()

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.88it/s]

CPU times: total: 234 ms
Wall time: 352 ms


In [14]:
print(f'We have {len(documents)} pages in total')

We have 9 pages in total


In [15]:
documents[8].page_content

'خبرنامه  ایمیلی  ما به روزرسان یهای  مربوط به محصولات جد ید،  پیشنهادات   انحصار ی  و نکات مف ید  مرتبط با  \nمحصولات ما را ارائه می دهد. می توانید در وبسایت ما مشترک خبرنامه شوید . \n \n \n \n آیا می توانم یک  محصول را در صورت تغییر نظر بازگردانم؟  \nپاسخ :   \nبله،  شما م ی توانید  ی ک  محصول را در صورت تغییر   نظر خود بازگردانید،   مشروط بر ا ینکه   محصول در شرایط   اولیه   و  \nبدون استفاده باشد. برا ی دستورالعمل های دقیق بازگشت، لطفاً به سیاست  بازگشت محصولات ما مراجعه کنید. \n '

# Splitter

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = CFG.split_chunk_size,
    chunk_overlap = CFG.split_overlap
)

texts = text_splitter.split_documents(documents)

print(f'We have created {len(texts)} chunks from {len(documents)} pages')

We have created 10 chunks from 9 pages


# Create Embeddings

In [18]:
%%time

### we create the embeddings only if they do not exist yet
if not os.path.exists(CFG.Embeddings_path + '/index.faiss'):

    ### download embeddings model
    embeddings = HuggingFaceInstructEmbeddings(
        model_name = CFG.embeddings_model_repo,
        model_kwargs = {"device": "cuda"}
    )

    ### create embeddings and DB
    vectordb = FAISS.from_documents(
        documents = texts, 
        embedding = embeddings
    )

    ### persist vector database
    vectordb.save_local(f"{CFG.Output_folder}/faiss_index_hp") # save in output folder
#     vectordb.save_local(f"{CFG.Embeddings_path}/faiss_index_hp") # save in input folder

load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: total: 500 ms
Wall time: 836 ms


# Load vector database

In [17]:
%%time

### download embeddings model
embeddings = HuggingFaceInstructEmbeddings(
    model_name = CFG.embeddings_model_repo,
    model_kwargs = {"device": "cuda"}
)

### load vector DB embeddings
vectordb = FAISS.load_local(
    CFG.Embeddings_path, # from input folder
#     CFG.Output_folder + '/faiss_index_hp', # from output folder
    embeddings
)

load INSTRUCTOR_Transformer
max_seq_length  512


ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).

In [19]:
### test if vector DB was loaded correctly
vectordb.similarity_search('حساب')

[Document(metadata={'source': 'Dataset2\\myQA.pdf', 'page': 5}, page_content='آیا می توانم یک  کالا را در سفارش خود تغییر دهم یا لغو کنم؟ \nپاسخ :'),
 Document(metadata={'source': 'Dataset2\\myQA.pdf', 'page': 8}, page_content='خبرنامه  ایمیلی  ما به روزرسان یهای  مربوط به محصولات جد ید،  پیشنهادات   انحصار ی  و نکات مف ید  مرتبط با  \nمحصولات ما را ارائه می دهد. می توانید در وبسایت ما مشترک خبرنامه شوید . \n \n \n \n آیا می توانم یک  محصول را در صورت تغییر نظر بازگردانم؟  \nپاسخ :   \nبله،  شما م ی توانید  ی ک  محصول را در صورت تغییر   نظر خود بازگردانید،   مشروط بر ا ینکه   محصول در شرایط   اولیه   و  \nبدون استفاده باشد. برا ی دستورالعمل های دقیق بازگشت، لطفاً به سیاست  بازگشت محصولات ما مراجعه کنید.'),
 Document(metadata={'source': 'Dataset2\\myQA.pdf', 'page': 0}, page_content="چگونه می توانم ی ک حساب کاربری  ایجاد کنم؟  \nپاسخ :   \nبرای   ایجاد  یک   حساب کاربری،  روی   دکمه  Sign Up' در گوشه بالا سمت راست وبسایت  ما کلیک  کنید  و  \nدستورالعمل ها را برای  تکمیل فرآیند ثبت نام د

# Prompt Template

- Custom prompt

In [20]:
prompt_template = """

سعی نکنید پاسخی بسازید، اگر نمی دانید فقط بگویید که نمی دانید.
به همان زبان سوال پرسیده شده پاسخ دهید.
برای پاسخ دادن به سوال در پایان فقط از متن زیر استفاده کنید.

{context}

Human: {question}
Assistant:"""


PROMPT = PromptTemplate(
    template = prompt_template, 
    input_variables = ["context", "question"]
)

In [21]:
# llm_chain = LLMChain(prompt=PROMPT, llm=llm)
# llm_chain

# Retriever chain

In [22]:
retriever = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever, 
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

In [23]:
### testing MMR search
question = "چگونه م یتوانم یک حساب کاربری ایجاد کنم ؟"
vectordb.max_marginal_relevance_search(question, k = CFG.k)

[Document(metadata={'source': 'Dataset2\\myQA.pdf', 'page': 5}, page_content='آیا می توانم یک  کالا را در سفارش خود تغییر دهم یا لغو کنم؟ \nپاسخ :'),
 Document(metadata={'source': 'Dataset2\\myQA.pdf', 'page': 0}, page_content="چگونه می توانم ی ک حساب کاربری  ایجاد کنم؟  \nپاسخ :   \nبرای   ایجاد  یک   حساب کاربری،  روی   دکمه  Sign Up' در گوشه بالا سمت راست وبسایت  ما کلیک  کنید  و  \nدستورالعمل ها را برای  تکمیل فرآیند ثبت نام دنبال کنید. \n \n \n \n چه روش های پرداختی  را قبول می کنید؟  \nپاسخ :   \nما  کارت های  اعتبار ی  اصلی،  کارتهای   بده ی  و  PayPal را به عنوان روشهای  پرداخت برای   سفارش های  آنل ی ن   \nقبول می کنیم. \n \n \n \n چگونه می توانم سفارش خود را پیگیری  کنم؟  \nپاسخ :   \nشما میتوانید با ورود به حساب کاربری خود و مراجعه به بخش  Order History' سفارش خود را پیگیری کنید. \nدر آنجا اطلعات پیگیری  مربوط به ارسال شما قرار دارد. \n \n \n \n سیاست  بازگشت کالاهای  شما چیست؟"),
 Document(metadata={'source': 'Dataset2\\myQA.pdf', 'page': 6}, page_content="اگر  نیاز  به تغی

# Post-process outputs

In [24]:
def wrap_text_preserve_newlines(text, width=700):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])
    
    sources_used = ' \n'.join(
        [
            source.metadata['source'].split('/')[-1][:-4]
            + ' - page: '
            + str(source.metadata['page'])
            for source in llm_response['source_documents']
        ]
    )
    
    ans = ans + '\n\nSources: \n' + sources_used
    return ans

In [25]:
def llm_ans(query):
    start = time.time()
    
    llm_response = qa_chain.invoke(query)
    ans = process_llm_response(llm_response)
    
    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

# Ask questions

In [26]:
CFG.model_name

'Persian'

In [28]:
query = "چگونه میتوانم یک حساب کاربری ایجاد کنم؟"
print(llm_ans(query))



سعی نکنید پاسخی بسازید، اگر نمی دانید فقط بگویید که نمی دانید.
به همان زبان سوال پرسیده شده پاسخ دهید.
برای پاسخ دادن به سوال در پایان فقط از متن زیر استفاده کنید.

آیا می توانم یک  کالا را در سفارش خود تغییر دهم یا لغو کنم؟
پاسخ :

چگونه می توانم ی ک حساب کاربری  ایجاد کنم؟
پاسخ :
برای   ایجاد  یک   حساب کاربری،  روی   دکمه  Sign Up' در گوشه بالا سمت راست وبسایت  ما کلیک  کنید  و
دستورالعمل ها را برای  تکمیل فرآیند ثبت نام دنبال کنید.



 چه روش های پرداختی  را قبول می کنید؟
پاسخ :
ما  کارت های  اعتبار ی  اصلی،  کارتهای   بده ی  و  PayPal را به عنوان روشهای  پرداخت برای   سفارش های  آنل ی ن
قبول می کنیم.



 چگونه می توانم سفارش خود را پیگیری  کنم؟
پاسخ :
شما میتوانید با ورود به حساب کاربری خود و مراجعه به بخش  Order History' سفارش خود را پیگیری کنید.
در آنجا اطلعات پیگیری  مربوط به ارسال شما قرار دارد.



 سیاست  بازگشت کالاهای  شما چیست؟

شما میتوانید از طر یق تلفن به شمارهیا  از طری ق ایمیل به آدرسیمیل]یم پشتیبانی مشتریان
ما تماس بگیرید. تیم ما در ساعات کار ی [ی].



 آیا خدمات ب

In [27]:
query = "آیا می توانم سفارش خود را تلفنی ثبت کنم؟"
print(llm_ans(query))



سعی نکنید پاسخی بسازید، اگر نمی دانید فقط بگویید که نمی دانید.
به همان زبان سوال پرسیده شده پاسخ دهید.
برای پاسخ دادن به سوال در پایان فقط از متن زیر استفاده کنید.

آیا می توانم یک  کالا را در سفارش خود تغییر دهم یا لغو کنم؟
پاسخ :

چگونه می توانم ی ک حساب کاربری  ایجاد کنم؟
پاسخ :
برای   ایجاد  یک   حساب کاربری،  روی   دکمه  Sign Up' در گوشه بالا سمت راست وبسایت  ما کلیک  کنید  و
دستورالعمل ها را برای  تکمیل فرآیند ثبت نام دنبال کنید.



 چه روش های پرداختی  را قبول می کنید؟
پاسخ :
ما  کارت های  اعتبار ی  اصلی،  کارتهای   بده ی  و  PayPal را به عنوان روشهای  پرداخت برای   سفارش های  آنل ی ن
قبول می کنیم.



 چگونه می توانم سفارش خود را پیگیری  کنم؟
پاسخ :
شما میتوانید با ورود به حساب کاربری خود و مراجعه به بخش  Order History' سفارش خود را پیگیری کنید.
در آنجا اطلعات پیگیری  مربوط به ارسال شما قرار دارد.



 سیاست  بازگشت کالاهای  شما چیست؟

متأسفانه،  ما سفارشهای  تلفن ی  را قبول نمی کنیم. لطفاً سفارش خود را از طر یق  وبسا یت   ما ثبت کن ید  تا تراکنش به
صورت امن و بدون مشکل انجام ش

In [31]:
query = "اگر بسته من گم شود یا آسیب ببیند چه کاری باید انجام دهم؟"
print(llm_ans(query))



سعی نکنید پاسخی بسازید، اگر نمی دانید فقط بگویید که نمی دانید.
به همان زبان سوال پرسیده شده پاسخ دهید.
برای پاسخ دادن به سوال در پایان فقط از متن زیر استفاده کنید.

آیا می توانم یک  کالا را در سفارش خود تغییر دهم یا لغو کنم؟
پاسخ :

چگونه می توانم ی ک حساب کاربری  ایجاد کنم؟
پاسخ :
برای   ایجاد  یک   حساب کاربری،  روی   دکمه  Sign Up' در گوشه بالا سمت راست وبسایت  ما کلیک  کنید  و
دستورالعمل ها را برای  تکمیل فرآیند ثبت نام دنبال کنید.



 چه روش های پرداختی  را قبول می کنید؟
پاسخ :
ما  کارت های  اعتبار ی  اصلی،  کارتهای   بده ی  و  PayPal را به عنوان روشهای  پرداخت برای   سفارش های  آنل ی ن
قبول می کنیم.



 چگونه می توانم سفارش خود را پیگیری  کنم؟
پاسخ :
شما میتوانید با ورود به حساب کاربری خود و مراجعه به بخش  Order History' سفارش خود را پیگیری کنید.
در آنجا اطلعات پیگیری  مربوط به ارسال شما قرار دارد.



 سیاست  بازگشت کالاهای  شما چیست؟

متأسفانه،  ما سفارشهای  تلفن ی  را قبول نمی کنیم. لطفاً سفارش خود را از طر یق  وبسا یت   ما ثبت کن ید  تا تراکنش به
صورت امن و بدون مشکل انجام ش

In [35]:
query = "آیا م یتوانم ی ک محصول را در صورت تغییر نظر بازگردانم ؟"
print(llm_ans(query))



سعی نکنید پاسخی بسازید، اگر نمی دانید فقط بگویید که نمی دانید.
به همان زبان سوال پرسیده شده پاسخ دهید.
برای پاسخ دادن به سوال در پایان فقط از متن زیر استفاده کنید.

آیا می توانم یک  کالا را در سفارش خود تغییر دهم یا لغو کنم؟
پاسخ :

چگونه می توانم ی ک حساب کاربری  ایجاد کنم؟
پاسخ :
برای   ایجاد  یک   حساب کاربری،  روی   دکمه  Sign Up' در گوشه بالا سمت راست وبسایت  ما کلیک  کنید  و
دستورالعمل ها را برای  تکمیل فرآیند ثبت نام دنبال کنید.



 چه روش های پرداختی  را قبول می کنید؟
پاسخ :
ما  کارت های  اعتبار ی  اصلی،  کارتهای   بده ی  و  PayPal را به عنوان روشهای  پرداخت برای   سفارش های  آنل ی ن
قبول می کنیم.



 چگونه می توانم سفارش خود را پیگیری  کنم؟
پاسخ :
شما میتوانید با ورود به حساب کاربری خود و مراجعه به بخش  Order History' سفارش خود را پیگیری کنید.
در آنجا اطلعات پیگیری  مربوط به ارسال شما قرار دارد.



 سیاست  بازگشت کالاهای  شما چیست؟

اگر  نیاز  به تغییر  یا   لغو  ی ک  کالا در سفارش خود دار ید،  لطفاً در اسرع وقت با تیم  پشتیبانی  مشتریان  ما تماس بگیرید.
ما شما را در مراحل ل

In [36]:
query = "چگونه میتوانم یک حساب کاربری ایجاد کنم؟"
print(llm_ans(query))



سعی نکنید پاسخی بسازید، اگر نمی دانید فقط بگویید که نمی دانید.
به همان زبان سوال پرسیده شده پاسخ دهید.
برای پاسخ دادن به سوال در پایان فقط از متن زیر استفاده کنید.

آیا می توانم یک  کالا را در سفارش خود تغییر دهم یا لغو کنم؟
پاسخ :

چگونه می توانم ی ک حساب کاربری  ایجاد کنم؟
پاسخ :
برای   ایجاد  یک   حساب کاربری،  روی   دکمه  Sign Up' در گوشه بالا سمت راست وبسایت  ما کلیک  کنید  و
دستورالعمل ها را برای  تکمیل فرآیند ثبت نام دنبال کنید.



 چه روش های پرداختی  را قبول می کنید؟
پاسخ :
ما  کارت های  اعتبار ی  اصلی،  کارتهای   بده ی  و  PayPal را به عنوان روشهای  پرداخت برای   سفارش های  آنل ی ن
قبول می کنیم.



 چگونه می توانم سفارش خود را پیگیری  کنم؟
پاسخ :
شما میتوانید با ورود به حساب کاربری خود و مراجعه به بخش  Order History' سفارش خود را پیگیری کنید.
در آنجا اطلعات پیگیری  مربوط به ارسال شما قرار دارد.



 سیاست  بازگشت کالاهای  شما چیست؟

شما میتوانید از طر یق تلفن به شمارهیا  از طری ق ایمیل به آدرسیمیل]یم پشتیبانی مشتریان
ما تماس بگیرید. تیم ما در ساعات کار ی [ی].



 آیا خدمات ب

In [37]:
query = "چه رو شهای پرداختی را قبول م یکنید؟"
print(llm_ans(query))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset




سعی نکنید پاسخی بسازید، اگر نمی دانید فقط بگویید که نمی دانید.
به همان زبان سوال پرسیده شده پاسخ دهید.
برای پاسخ دادن به سوال در پایان فقط از متن زیر استفاده کنید.

آیا می توانم یک  کالا را در سفارش خود تغییر دهم یا لغو کنم؟
پاسخ :

چگونه می توانم ی ک حساب کاربری  ایجاد کنم؟
پاسخ :
برای   ایجاد  یک   حساب کاربری،  روی   دکمه  Sign Up' در گوشه بالا سمت راست وبسایت  ما کلیک  کنید  و
دستورالعمل ها را برای  تکمیل فرآیند ثبت نام دنبال کنید.



 چه روش های پرداختی  را قبول می کنید؟
پاسخ :
ما  کارت های  اعتبار ی  اصلی،  کارتهای   بده ی  و  PayPal را به عنوان روشهای  پرداخت برای   سفارش های  آنل ی ن
قبول می کنیم.



 چگونه می توانم سفارش خود را پیگیری  کنم؟
پاسخ :
شما میتوانید با ورود به حساب کاربری خود و مراجعه به بخش  Order History' سفارش خود را پیگیری کنید.
در آنجا اطلعات پیگیری  مربوط به ارسال شما قرار دارد.



 سیاست  بازگشت کالاهای  شما چیست؟

متأسفانه،  ما سفارشهای  تلفن ی  را قبول نمی کنیم. لطفاً سفارش خود را از طر یق  وبسا یت   ما ثبت کن ید  تا تراکنش به
صورت امن و بدون مشکل انجام ش

In [38]:
query = "چه روش های پرداختی را قبول می کنید؟"
print(llm_ans(query))



سعی نکنید پاسخی بسازید، اگر نمی دانید فقط بگویید که نمی دانید.
به همان زبان سوال پرسیده شده پاسخ دهید.
برای پاسخ دادن به سوال در پایان فقط از متن زیر استفاده کنید.

آیا می توانم یک  کالا را در سفارش خود تغییر دهم یا لغو کنم؟
پاسخ :

چگونه می توانم ی ک حساب کاربری  ایجاد کنم؟
پاسخ :
برای   ایجاد  یک   حساب کاربری،  روی   دکمه  Sign Up' در گوشه بالا سمت راست وبسایت  ما کلیک  کنید  و
دستورالعمل ها را برای  تکمیل فرآیند ثبت نام دنبال کنید.



 چه روش های پرداختی  را قبول می کنید؟
پاسخ :
ما  کارت های  اعتبار ی  اصلی،  کارتهای   بده ی  و  PayPal را به عنوان روشهای  پرداخت برای   سفارش های  آنل ی ن
قبول می کنیم.



 چگونه می توانم سفارش خود را پیگیری  کنم؟
پاسخ :
شما میتوانید با ورود به حساب کاربری خود و مراجعه به بخش  Order History' سفارش خود را پیگیری کنید.
در آنجا اطلعات پیگیری  مربوط به ارسال شما قرار دارد.



 سیاست  بازگشت کالاهای  شما چیست؟

شما میتوانید از طر یق تلفن به شمارهیا  از طری ق ایمیل به آدرسیمیل]یم پشتیبانی مشتریان
ما تماس بگیرید. تیم ما در ساعات کار ی [ی].



 آیا خدمات ب